In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib
import chart
import radarkit

In [ ]:
# matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rc('text', usetex=False)

In [ ]:
sweep = radarkit.read('data/PX-20180308-043920-E4.5-Z.nc')
z = sweep['moments']['Z']

In [ ]:
z.shape

In [ ]:
# a, r = np.mgrid[slice(0, 2.0 * np.pi,  np.pi / 180.0), slice(0, 60, 0.12)]
a, r = np.mgrid[slice(0, 2.0 * np.pi,  np.pi / 180.0), slice(0, 1145)]

r = r * 0.12;
x = r * np.sin(a)
y = r * np.cos(a)

In [ ]:
ppi = chart.showPPI(x, y, z)